In [1]:
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
from gym import spaces
from pypylon import pylon
import stable_baselines
import time
from matplotlib import cm
from stable_baselines.common.schedules import LinearSchedule
%matplotlib inline

env_closer = closer.Closer()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class CorningEnv(object):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    

    
    def __init__(self, camera, 
                     GainRaw = 36, 
                     ExposureTimeRaw = 20000, 
                     AcquisitionFrameRateAbs = 10,
                     Width = 417,
                     Height = 404,
                     threshold = 85):
        
        ##############camera##############
        
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        
        self.camera = camera
        
        # https://docs.baslerweb.com/features
        self.camera.GainRaw.Value = GainRaw
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        self.camera.AcquisitionFrameRateAbs.Value = AcquisitionFrameRateAbs
        
        # get width and height from camera for making obs space
        self.camera.Width.Value = Width
        self.camera.Height.Value = Height
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        
        
        ##############lens###############
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        ##############env action and observation##############
        # Focus Voltage
        self.action_space = spaces.Discrete(69)
        # image
        self.observation_space = spaces.Box(low=0, high=255, 
                                             shape=(Height, Width, 3),dtype = np.uint8)
        
        
        ##############for grabbing images from camera##############
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        ##############YOLO##############
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        ########################################################
        self.threshold = threshold
        self.file_path = r'C:\Users\CIG\Documents\imgs_RL'
        self.i = 0
        time.sleep(1)
        
#     def compute_reward(self, achieved_goal, desired_goal, info):
#         """Compute the step reward. This externalizes the reward function and makes
#         it dependent on a desired goal and the one that was achieved. If you wish to include
#         additional rewards that are independent of the goal, you can include the necessary values
#         to derive it in 'info' and compute it accordingly.
#         Args:
#             achieved_goal (object): the goal that was achieved during execution
#             desired_goal (object): the desired goal that we asked the agent to attempt to achieve
#             info (dict): an info dictionary with additional information
#         Returns:
#             float: The reward that corresponds to the provided achieved goal w.r.t. to the desired
#             goal. Note that the following should always hold true:
#                 ob, reward, done, info = env.step()
#                 assert reward == env.compute_reward(ob['achieved_goal'], ob['goal'], info)
#         """
#         raise NotImplementedError
        
        
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    def focus_value(self, im):
        # Calculate the gradient
        sobelx = cv2.Sobel(np.float32(im), cv2.CV_64F ,1 , 0, ksize=5)
        sobely = cv2.Sobel(np.float32(im),cv2.CV_64F,0,1,ksize=5)

        abs_sobel_x = cv2.convertScaleAbs(sobelx) # converting back to uint8
        abs_sobel_y = cv2.convertScaleAbs(sobely)
        #print(abs_sobel_x )

        # Combine the two gradients with equal weight
        dst = cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)
        #print(dst)

        # Calculate the average gradient for the image
        # I convert it to a numpy array for ease of calculation
        return pl.asarray(dst).mean()
    
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        #time.sleep(5)
        
    def grab_image(self):
        #https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
#                 print(img[0])
#                 print('shape:', img.shape)
#                 plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
    
    def step(self, action):
        
        print('real action:', action)
        if action < 24:
            action = 24
            
        self.lens_movement(action)
        
        camera_observation = self.grab_image()
        prediction_yolo = self.model_yolo(camera_observation)
        final_result_yolo = self.find_class(prediction_yolo)
        
        if final_result_yolo.size == 0:
            foc_value = -1
            reward = -10
            
        else:
            image = Image.fromarray(camera_observation)
            image = image.crop((final_result_yolo[0], 
                             final_result_yolo[1], 
                             final_result_yolo[2], 
                             final_result_yolo[3]))
            foc_value = self.focus_value(image)
            reward = -1
            
        if foc_value > self.threshold: #or self.i == 10:
            done = True
            print(True, action, self.i)
        else:
            done = False
            print(False, action)
            
        im = Image.fromarray(camera_observation)
        filename = self.file_path + "\saved_pypylon_img_%s.png" % (str(action))
        im.save(filename)
        
        self.i+=1
        #print(camera_observation)
        camera_observation = camera_observation.astype(np.uint8)
            
        return camera_observation, reward, done, {}
  

    def reset(self):
        """Resets the environment to an initial state and returns an initial
        observation.
        Note that this function should not reset the environment's random
        number generator(s); random variables in the environment's state should
        be sampled independently between multiple calls to `reset()`. In other
        words, each call of `reset()` should yield an environment suitable for
        a new episode, independent of previous episodes.
        Returns:
            observation (object): the initial observation.
        """
        x = c_double(24)
        self.lib.Casp_SetFocusVoltage(x)
        self.i = 0
        time.sleep(1)
        return self.grab_image()


    def close(self):
        """Override close in your subclass to perform any necessary cleanup.
        Environments will automatically close() themselves when
        garbage collected or when the program exits.
        """
        pass



In [3]:
# Create an instant camera object with the camera device found first.
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

In [4]:
env = CorningEnv(camera)

417 404
eCOMCaspErr: 0 0


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   

Adding autoShape... 


In [5]:
# TIMESTEPS = 500
# sched_LR = LinearSchedule(TIMESTEPS, 0.005, 0.00001)

model = PPO2(MlpPolicy, env, 
             #learning_rate = sched_LR.value,
             learning_rate = 0.1,
             verbose=1, 
             tensorboard_log = '')

Wrapping the env in a DummyVecEnv.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:191: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:200: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\policies.py:116: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\input.py:25: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:206: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:240: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:242: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



In [6]:
model.learn(total_timesteps=200, tb_log_name="ppo2_")

From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\base_class.py:1169: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



real action: 7
False 24
real action: 47
False 47
real action: 39
False 39
real action: 38
False 38
real action: 57
False 57
real action: 67
False 67
real action: 7
False 24
real action: 5
False 24
real action: 30
False 30
real action: 63
False 63
real action: 63
False 63
real action: 12
False 24
real action: 48
False 48
real action: 10
False 24
real action: 21
False 24
real action: 34
False 34
real action: 53
False 53
real action: 58
False 58
real action: 59
False 59
real action: 49
False 49
real action: 6
False 24
real action: 65
False 65
real action: 35
False 35
real action: 26
False 26
real action: 18
False 24
real action: 14
False 24
real action: 31
False 31
real action: 11
False 24
real action: 45
False 45
real action: 28
False 28
real action: 38
False 38
real action: 15
False 24
real action: 63
False 63
real action: 27
False 27
real action: 13
False 24
real action: 47
False 47
real action: 59
False 59
real action: 1
False 24
real action: 58
False 58
real action: 3
False 24
real a

In [6]:
model.learn(total_timesteps=1000, tb_log_name="ppo2_uint")

From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\base_class.py:1169: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
#tensorboard --logdir=C:\\Users\\CIG\\Documents\\MATLAB

In [7]:
camera.Close()